### 1. event 발생 시 websocket과 연결하여 event 포착한다.

In [1]:
%%writefile src/webSocketTest.js
var Web3 = require('web3');
const myProvider = new Web3.providers.WebsocketProvider("ws://localhost:8345", {
    clientConfig: {
        keepalive:true, keepaliveInterval:10000
    } 
  });
var web3 = new Web3(myProvider);
console.log("(1) websocket url: ", myProvider.connection.url); //web3.currentProvider.connection.url
myProvider.on('connect', function() {
    console.log("(2) connecting websocket: "+web3.currentProvider.connected);
    //myProvider.disconnect();
    web3.currentProvider.connection.close();
    console.log("(3) disconnecting Websocket: "+web3.currentProvider.connected);
});
myProvider.on('close', function() { console.log("--> Websocket closed"); });
myProvider.on('end', function() { console.log("--> Websocket ended"); });
myProvider.on('error', function(error) { console.error(error); });

Writing src/webSocketTest.js


In [2]:
!node src/webSocketTest.js

(1) websocket url:  ws://localhost:8345
(2) connecting websocket: true
(3) disconnecting Websocket: false
--> Websocket ended
--> Websocket closed


In [3]:
%%writefile src/EventTest.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;

contract EventTest {
    event MyLog(string my);
    function myFunction() public {
        emit MyLog("Hello World!");
    }
}

Writing src/EventTest.sol


### 배포는 http로 가능

In [5]:
!solc --optimize --combined-json abi,bin src/EventTest.sol > src/EventTest.json

In [6]:
!cat src/EventTest.json

{"contracts":{"src/EventTest.sol:EventTest":{"abi":[{"anonymous":false,"inputs":[{"indexed":false,"internalType":"string","name":"my","type":"string"}],"name":"MyLog","type":"event"},{"inputs":[],"name":"myFunction","outputs":[],"stateMutability":"nonpayable","type":"function"}],"bin":"608060405234801561001057600080fd5b5060c28061001f6000396000f3fe6080604052348015600f57600080fd5b506004361060285760003560e01c8063c3780a3a14602d575b600080fd5b60336035565b005b7f5186edd9beca61d795526ca1f274260b3fc74be3e10e1f02e1be1552e14f13736040516082906020808252600c908201526b48656c6c6f20576f726c642160a01b604082015260600190565b60405180910390a156fea2646970667358221220dc65f0285737028f1aab08a2f886f83ab92a51ec712b3b04a344a01efb90336d64736f6c634300080c0033"}},"version":"0.8.12+commit.f00d7308.Darwin.appleclang"}


In [7]:
%%writefile src/EventTestDeployFromFile.js
var Web3 = require('web3');
var _abiBinJson = require('./EventTest.json');      //importing a javascript file

var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

contractName=Object.keys(_abiBinJson.contracts); // reading ['src/EventTest.sol:EventTest']
console.log("- contract name: ", contractName);
_abi=_abiBinJson.contracts[contractName].abi;
_abiArray=JSON.parse(JSON.stringify(_abi));
//_abiArray=JSON.parse(_abi);      //JSON parsing needed!! //SyntaxError: Unexpected token o in JSON at position 1
_bin=_abiBinJson.contracts[contractName].bin;

//console.log("- ABI: " + _abiArray);
//console.log("- Bytecode: " + _bin);

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: "0x"+_bin})
        .send({from: accounts[0], gas: 259210}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance){
        //    console.log(newContractInstance)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Writing src/EventTestDeployFromFile.js


In [ ]:
%%writefile src/EventTestDeployGeth.js
var Web3 = require('web3');
var _abiBinJson = require('./EventTest.json');

var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8445"));

contractName = Object.keys(_abiBinJson.contracts);
console.log("- contract name: ", contractName);
_abi=_abiBinJson.contracts[contractName].abi;
_abiArray=JSON.parse(JSON.stringify(_abi));
_bin=_abiBinJson.contracts[contractName].bin;
web3.personal.unlockAccount(web3.eth.accounts[0],'password');
async finction deploy() {
    const accounts = await web3.eth.accounts;
}


In [8]:
!node src/EventTestDeployFromFile.js

- contract name:  [ 'src/EventTest.sol:EventTest' ]
Deploying the contract from 0x92D6b70ee4483106Ae97537Ac0b2B45a67C732ED
hash: 0x8388dc49617c6bd3b173f2441368fec3c9646694352f6e897b6bf654af66e1f3
---> The contract deployed to: 0x18a453CfB01bf2C5B004C0c2CEE8c320Fe538a00


In [9]:
%%writefile src/EventTestHttpNoEventFiredUse.js
var Web3=require('web3');
var _abiBinJson = require('./EventTest.json');      //importing a javascript file

var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345")); // Http provider...
contractName=Object.keys(_abiBinJson.contracts); // reading ['src/EventTest.sol:EventTest']
//console.log("- contract name: ", contractName); //or console.log(contractName[0]);
_abiArray=_abiBinJson.contracts[contractName].abi; //use just as read from file
//_abiArray=JSON.parse(JSON.stringify(_abi));
//_abiArray=JSON.parse(_abi);      //JSON parsing needed!!
//_bin=_abiBinJson.contracts[contractName].bin;
console.log("- ABI: " + _abiArray);
//console.log("- Bytecode: " + _bin);
var _test = new web3.eth.Contract(_abiArray,"0x8911bA097c812Bf0B3ff22F90eaf2A905112C5a6");
var event = _test.events.MyLog({fromBlock: 0}, function (error, result) {
    if (!error) {
        console.log("Event fired: " + JSON.stringify(result.returnValues));
    }
});

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    const value = await _test.methods.myFunction()
        .send({from: accounts[0], gas: 364124, gasPrice: '1000000000'})
        //.then(function(value) {console.log("---> myFunction called " + JSON.stringify(value.events.MyLog.returnValues));});
    console.log("---> myFunction called " + JSON.stringify(value.events.MyLog.returnValues));
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
}
doIt()

Writing src/EventTestHttpNoEventFiredUse.js


In [20]:
%%writefile src/EventTestWsUse.js
var Web3=require('web3');
var fs = require('fs');
var _abiBinJson = require('./EventTest.json');      //importing a javascript file

//var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
//var web3 = new Web3(new Web3.providers.WebsocketProvider("http://localhost:8345"));
var web3 = new Web3(new Web3.providers.WebsocketProvider("ws://localhost:8345"));
contractName=Object.keys(_abiBinJson.contracts); // reading ['src/EventTest.sol:EventTest']
//console.log("- contract name: ", contractName); //or console.log(contractName[0]);
_abiArray=_abiBinJson.contracts[contractName].abi; //use just as read from file
//_abiArray=JSON.parse(JSON.stringify(_abi));
//_abiArray=JSON.parse(_abi);      //JSON parsing needed!!
//_bin=_abiBinJson.contracts[contractName].bin;
console.log("- ABI: " + _abiArray);
//console.log("- Bytecode: " + _bin);
//var _test = new web3.eth.Contract(_abiArray,"0x8911bA097c812Bf0B3ff22F90eaf2A905112C5a6");
//var event = _test.events.MyLog({fromBlock: 0}, function (error, result) {
//    if (!error) {
//        console.log("Event fired: " + JSON.stringify(result.returnValues));
//    }
//});

async function doIt() {
    var _test = new web3.eth.Contract(_abiArray, '0x18a453CfB01bf2C5B004C0c2CEE8c320Fe538a00');
    var event = _test.events.MyLog({fromBlock: 0}, function (error, result) {
        if (!error) {
            log = JSON.stringify(result.returnValues);
            console.log("Event fired: " + log);
            //fs.writeFile("src/EventTestLog.txt", log, "utf-8", function(e) {
            fs.appendFile("src/EventTestLog.txt", log, "utf-8", function(e) {
                if(!e) {
                    console.log(">> Writing to file");
                }
            });
        }
    });
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    const value = await _test.methods.myFunction()
        .send({from: accounts[0], gas: 364124, gasPrice: '1000000000'})
        //.then(function(value) {console.log("---> myFunction called " + JSON.stringify(value.events.MyLog.returnValues));});
    console.log("---> myFunction called " + JSON.stringify(value.events.MyLog.returnValues));
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
    process.exit(1); //force exit to disconnect websocket
}

doIt()

Overwriting src/EventTestWsUse.js


In [21]:
!node src/EventTestWsUse.js

- ABI: [object Object],[object Object]
Account: 0x92D6b70ee4483106Ae97537Ac0b2B45a67C732ED
Event fired: {"0":"Hello World!","my":"Hello World!"}
Balance before: 999999571085625000000
>> Writing to file
Event fired: {"0":"Hello World!","my":"Hello World!"}
---> myFunction called {"0":"Hello World!","my":"Hello World!"}
>> Writing to file
Balance after: 999999548270625000000
Balance diff: 22814999969792


In [22]:
!cat src/EventTestLog.txt

{"0":"Hello World!","my":"Hello World!"}{"0":"Hello World!","my":"Hello World!"}{"0":"Hello World!","my":"Hello World!"}

In [23]:
%%writefile src/FallbackTest.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;

contract FallbackTest {
    event PrintLog(string);
    function callA () pure public returns(string memory){
        return "doing callA";
    }
    fallback () external {
        emit PrintLog("fallback called");
    }
}

Writing src/FallbackTest.sol


In [24]:
!solc --optimize --combined-json abi,bin src/FallbackTest.sol > src/FallbackTest.json

In [26]:
!cat src/FallbackTest.json

{"contracts":{"src/FallbackTest.sol:FallbackTest":{"abi":[{"anonymous":false,"inputs":[{"indexed":false,"internalType":"string","name":"","type":"string"}],"name":"PrintLog","type":"event"},{"stateMutability":"nonpayable","type":"fallback"},{"inputs":[],"name":"callA","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"pure","type":"function"}],"bin":"608060405234801561001057600080fd5b50610147806100206000396000f3fe608060405234801561001057600080fd5b506004361061002b5760003560e01c8063e7f09e0514610086575b7f968f0302429ff0e5bd56a45ce3ba1f4fa79f4b822857e438616435f00c3b59fd60405161007c906020808252600f908201526e19985b1b189858dac818d85b1b1959608a1b604082015260600190565b60405180910390a1005b604080518082018252600b81526a646f696e672063616c6c4160a81b602082015290516100b391906100bc565b60405180910390f35b600060208083528351808285015260005b818110156100e9578581018301518582016040015282016100cd565b818111156100fb576000604083870101525b50601f01601f191692909201604001939250505056fea26

In [27]:
%%writefile src/FallbackTestDeployFromFile.js
var Web3 = require('web3');
var _abiBinJson = require('./FallbackTest.json');      //importing a javascript file
//var fs=require('fs');
//var _str = fs.readFileSync("src/FallbackTest.json");
//var _json=JSON.parse(_str)

var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

contractName=Object.keys(_abiBinJson.contracts); // reading ['src/FallbackTest.sol:FallbackTest']
console.log("- contract name: ", contractName);
_abiArray=_abiBinJson.contracts[contractName].abi;
//_abiArray=JSON.parse(JSON.stringify(_abi));
//_abiArray=JSON.parse(_abi);      //JSON parsing needed!! //SyntaxError: Unexpected token o in JSON at position 1
_bin=_abiBinJson.contracts[contractName].bin;

//console.log("- ABI: " + _abiArray);
//console.log("- Bytecode: " + _bin);

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: "0x"+_bin})
        .send({from: accounts[0], gas: 1000000}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance){
        //    console.log(newContractInstance)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()


Writing src/FallbackTestDeployFromFile.js


In [28]:
!node src/FallbackTestDeployFromFile.js

- contract name:  [ 'src/FallbackTest.sol:FallbackTest' ]
Deploying the contract from 0x92D6b70ee4483106Ae97537Ac0b2B45a67C732ED
hash: 0x400e1f35c1027239491dcc219f7b69ed7d5e2e2b855fc9f9df429bad40565402
---> The contract deployed to: 0xbb812Dbd0dC65CD50d5f2A702833631a8761Cf26


In [51]:
%%writefile src/FallbackTestUseFromFile.js
var Web3=require('web3');
//var _abiJson = require('./FallbackTestABI.json');

var _abiBinJson = require('./FallbackTest.json');      //importing a javascript file

//var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
//var web3 = new Web3(new Web3.providers.WebsocketProvider("http://localhost:8345"));
var web3 = new Web3(new Web3.providers.WebsocketProvider("ws://localhost:8345"));
contractName=Object.keys(_abiBinJson.contracts); // reading ['src/EventTest.sol:EventTest']
//console.log("- contract name: ", contractName); //or console.log(contractName[0]);
_abiArray=_abiBinJson.contracts[contractName].abi; //use just as read from file
console.log("- ABI: " + _abiArray);


var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    web3 = new Web3(new Web3.providers.WebsocketProvider("ws://117.16.44.45:8345"));
    //web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8345"));
}

// contractName=Object.keys(_abiJson.contracts); // reading ['src//FallbackTest.sol:FallbackTest']
// console.log("- contract name: ", contractName[0]); //or console.log(contractName);
//_abiArray=JSON.parse(_abiJson.contracts[contractName].abi);    //JSON parsing needed!!
//_bin=_binJson.contracts[contractName].bin;
//console.log("- ABI: " + _abiArray);
//console.log("- Bytecode: " + _bin);

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    var _instance = new web3.eth.Contract(_abiArray, "0xbb812Dbd0dC65CD50d5f2A702833631a8761Cf26");
    var event = _instance.events.PrintLog(function (error, result) {
        if (!error) {
            console.log("Event fired: " + JSON.stringify(result) + "\n---> " + JSON.stringify(result.returnValues));
        }
    });

    _instance.methods.callA().call().then(function(res) { console.log(res); });  //null
    //call without calling any method
    await _instance.methods.callB().send({from:accounts[0], to: "0x3991e87b71cBFf94aA0718F341d8Ad4bCF969f36"}); //fail
    //await _instance.methods.callA().send({from:accounts[0], data:"0x1234"});  //empty calldata to call fallback -> fail
    web3.eth.sendTransaction({from:accounts[0], to: "0x3991e87b71cBFf94aA0718F341d8Ad4bCF969f36"}); //fallback called
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
    process.exit(1); //force exit to disconnect websocket

}

doIt()

Overwriting src/FallbackTestUseFromFile.js


In [52]:
!node src/FallbackTestUseFromFile.js

- ABI: [object Object],[object Object],[object Object]
Account: 0x92D6b70ee4483106Ae97537Ac0b2B45a67C732ED
Balance before: 999998963155311254111
/Users/broiron/Code/201710864/src/FallbackTestUseFromFile.js:44
    await _instance.methods.callB().send({from:accounts[0], to: "0x3991e87b71cBFf94aA0718F341d8Ad4bCF969f36"}); //fail
                            ^

TypeError: _instance.methods.callB is not a function
    at doIt (/Users/broiron/Code/201710864/src/FallbackTestUseFromFile.js:44:29)
    at processTicksAndRejections (node:internal/process/task_queues:96:5)

Node.js v17.8.0
